In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [2]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""
        You are an expert editor focused on clarity and simplicity.
        Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
        Don't worry about persuasion or tone - just make the message easy to read and understand.
    """,
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""
        You are a communication coach focused on emotional tone and professionalism.
        Your job is to make the email sound warm, confident, and human - while staying professional
        and appropriate for the audience. Improve the emotional resonance, polish the phrasing,
        and adjust any words that my come off as stiff, cold, or overly casual.
    """,
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""
        You are persuasion expert trained in marketing, behavioral psychology,
        and copywriting. Your job is to enhance the email's persuasive power: imporve call to action,
        structure arguments, and emphasize benefits. Remove weak or passive language.
    """,
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""
        You are an advanced email-writing specialist.
        Your role is to read all prior agent responses and revisions, and then **synthesize the best ideas** into a unified,
        polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements;
        Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, effective, and readable.
    """,
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""
        You are an email quality evaluator. Your job is to perform a final review of the synthesized email
        and determine if it meets professional standards. Review the email for:
        Clarity and flow, appropriate professional tone, effective call-to-cation, and overall coherence.
        Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone,
        or missing key elements), provide ONE specific improvement suggestion. If the email
        meets professional standards and communicates effectively, respond with 'The email meets professional
        standards.' followed by 'TERMINATE' on a new line. You should approve emails that are
        good enough for professional use, even if not perfect.
    """,
)

In [3]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [4]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! I'm hungry, buy me lunch and invest in my business. Thanks"
    )
)

---------- TextMessage (user) ----------
Hi! I'm hungry, buy me lunch and invest in my business. Thanks
---------- TextMessage (ClarityAgent) ----------
Hi! I'm hungry. Please buy me lunch and invest in my business. Thanks.
---------- TextMessage (ToneAgent) ----------
Subject: A Little Help to Fuel Both My Lunch and My Business

Hi [Recipient's Name],

I hope this message finds you well! I wanted to reach out because I’m feeling a bit peckish today, and I would truly appreciate your support in getting lunch. 

Additionally, I’m at a stage where I’m looking to move my business forward, and I would love to explore any possibilities for investment or guidance you might be able to offer. Your insight would mean a lot to me.

Thank you so much for considering my requests! Looking forward to hearing from you.

Warm regards,  
[Your Name]  
[Your Contact Information]
---------- TextMessage (PersuasionAgent) ----------
Subject: Fuel My Day: A Lunch Break and a Business Opportunity

Hi [Recipi

TaskResult(messages=[TextMessage(id='4d45a251-a3d8-4f18-af22-428c9dabae30', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 25, 5, 38, 21, 915759, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry, buy me lunch and invest in my business. Thanks", type='TextMessage'), TextMessage(id='baf23fad-16f3-4ba6-9e5c-34eed0973a2c', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=77, completion_tokens=17), metadata={}, created_at=datetime.datetime(2025, 11, 25, 5, 38, 22, 758565, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry. Please buy me lunch and invest in my business. Thanks.", type='TextMessage'), TextMessage(id='151f9885-b2ce-4c99-b7e5-0aa19d6721b6', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=122, completion_tokens=131), metadata={}, created_at=datetime.datetime(2025, 11, 25, 5, 38, 26, 107380, tzinfo=datetime.timezone.utc), content="Subject: A Little Help to Fuel Both My Lunch and My Business\n\nHi [Recipien